In [1]:
#START_HERE
#Notebook serves as tutorial.


In [ ]:
#START
#CELL 1
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd
import s3fs

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

#Garbage Collector
import gc 

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
#import seaborn as sns    NOTE: Will have to install seaborn later
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score, roc_curve, auc 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

#To read in files
fs = s3fs.S3FileSystem(anon=False)

In [2]:
#CELL 2
#Read in application_train.csv 
with fs.open('group12credits/application_train.csv') as f:
    train_df = pd.read_csv(f)  

In [4]:
#CELL 3
#Get the ids in application train
valid_IDs=train_df['SK_ID_CURR'].copy()
valid_IDs.sort()

#Get the ids and target values from application_train 
bareBones=train_df[['SK_ID_CURR','TARGET']].copy()

#Delete train_df and call gc.collect() to avoid memory errors 
#import gc 
#del train_df 
#gc.collect()

In [5]:
#CELL 4
#Read in the other csv file (previous_application.csv)
with fs.open('group12credits/previous_application.csv') as f:
    prevApps_all = pd.read_csv(f)
    
#Filter out the rows in the other csv file that are not part of application_train so there is less memory used 
#Sort values by the IDs for easier processing 
filteredApps=prevApps_all.loc[prevApps_all['SK_ID_CURR'].isin(valid_IDs)].sort_values(by=['SK_ID_CURR'])

#delete data frames we are not using anymore avoid memory leaks, call garbage collector 
#import gc
del prevApps_all, valid_IDs
gc.collect()

579

In [24]:
#CELL 5
#Merge or inner join bareBones and filtered Apps together 
prevApps=filteredApps.merge(bareBones,on='SK_ID_CURR',how='inner')

In [25]:
#CELL 6

#Set csv_df to merged file to perform pre-processing needed 
#Note: This will be used to check how each of the columns correlates to the Target variable. 
#The columns with the highest correlations will be added in one at a time to check if this improves our ROC AUC Curve/Classifier

csv_df=prevApps

#Avoid memory leaks 
del prevApps
gc.collect()

31

In [6]:
#CELL 7
#Check number of columns before pre-processing:
len(csv_df.columns)

124

In [7]:
#CELL 8
# Create a label encoder object to perform label encoding on objects with 2 or fewer unique catergories.(i.e.sex@birth 1=female, 0=male)

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in csv_df:
    if csv_df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(csv_df[col].unique())) <= 2:
            # Train on the training data
            le.fit(csv_df[col])
            # Transform both training and testing data  NOTE: Removed testing data
            csv_df[col] = le.transform(csv_df[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [8]:
#CELL 9
# one-hot encoding of categorical variables
csv_df = pd.get_dummies(csv_df)

In [34]:
#CELL 10

#Replace all spaces with underscores
csv_df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

# Find correlations with the target and sort (takes a minute to compile)
correlations = csv_df.corr()['TARGET'].sort_values()

In [35]:
#CELL 11

#Find how each column correlates to target 

import re
def interpretCorrelations(c):
    for line in c:
        title = line.split(' ')[0]
        status = ""
        try:
            num = abs(float(list(filter(None, line.split(' ')))[1]))
            if num <= .19:
                status = "very weak"
            elif num <= .39:
                status = "weak"
            elif num <= .59:
                status = "moderate"
            elif num <= .79:
                status = "strong"
            elif num <= 1:
                status = "very strong"
            #print(title + "\t" + str(num) + "\t" + status)
            print("Title: {0:30} Correlation: {1:5}\tStatus: {2:50}".format(title, num, status))
        except:
            print("An error occurred parsing the correlation coefficient")
            pass
        
    
# Display correlations
interpretCorrelations(correlations.to_string().splitlines())

Title: CODE_REJECT_REASON_XAP         Correlation: 0.052015	Status: very weak                                         
Title: NAME_CONTRACT_STATUS_Approved  Correlation: 0.049161	Status: very weak                                         
Title: DAYS_FIRST_DRAWING             Correlation: 0.031154	Status: very weak                                         
Title: NAME_PORTFOLIO_POS             Correlation: 0.030791	Status: very weak                                         
Title: NAME_CONTRACT_TYPE_Consumer_loans Correlation: 0.02994	Status: very weak                                         
Title: HOUR_APPR_PROCESS_START        Correlation: 0.027809	Status: very weak                                         
Title: NAME_YIELD_GROUP_low_normal    Correlation: 0.026615	Status: very weak                                         
Title: RATE_DOWN_PAYMENT              Correlation: 0.026111	Status: very weak                                         
Title: NAME_PAYMENT_TYPE_Cash_through_the_bank

In [9]:
#CELL 12
#Check number of columns after pre-processing
len(csv_df.columns)

254

In [ ]:
#CELL 13
#Columns from Previous_Application to use:

#Postive Correlations
#Title: CODE_REJECT_REASON_XAP         Correlation: 0.052015	Status: very weak                                         
#Title: NAME_CONTRACT_STATUS_Approved  Correlation: 0.049161	Status: very weak                                         
#Title: DAYS_FIRST_DRAWING             Correlation: 0.031154	Status: very weak                                         
#Title: NAME_PORTFOLIO_POS             Correlation: 0.030791	Status: very weak  

#Negative Correlations:
#Title: CODE_REJECT_REASON_HC          Correlation: 0.037158	Status: very weak                                         
#Title: CODE_REJECT_REASON_SCOFR       Correlation: 0.038432	Status: very weak                                         
#Title: DAYS_DECISION                  Correlation: 0.039901	Status: very weak                                         
#Title: NAME_PRODUCT_TYPE_walk-in      Correlation: 0.042842	Status: very weak                                         
#Title: NAME_CONTRACT_STATUS_Refused   Correlation: 0.054458	Status: very weak  

#'CODE_REJECT_REASON'

In [ ]:
#CELL 14
#NOTE BOOK PART TWO:
#Read in application train, add in column from previous application, perform pre-processing then check if ROC_AUC Curve improves.
del correlations, csv_df 

In [5]:
#CELL 15
#Re-run cells 1-4 to read in both application_train and previous_app (REMEMBER NOT TO CALL THE GARBAGE COLLECTOR ON application_train this time since we still need it)
#Merge or inner join the additional columns from filtered Apps and train_df together 
additional_Columns=filteredApps[['SK_ID_CURR','NAME_CONTRACT_STATUS','CODE_REJECT_REASON']]
csv_df=train_df.merge(additional_Columns,on='SK_ID_CURR',how='inner')

#No longer need filteredApps and train_df
del filteredApps,train_df
gc.collect()

#Re-run cells 7,8,9 & 12 (label-encoding and pre-processing (also length of columns check if desired),
#then run the rest of the cell, (this takes a while)
#See if ROC_AUC curve has improved, keept track of findings, repeat 

52

In [10]:
#CELL 16
#Lets run our classifier to check the ROC_AUC curve before adding in any extra columns. 
#Split csvfile into train and test. 
X_train, X_valid, y_train, y_valid = train_test_split(
    csv_df.drop(['TARGET','SK_ID_CURR'], axis=1), csv_df['TARGET'],
    test_size=0.3, random_state=17)

In [11]:
#CELL 17
#Run lightGBM 
lgb_clf=LGBMClassifier(random_state=17)
lgb_clf.fit(X_train,y_train)

#Avoid memory errors
#del lgb_clf
#gc.collect()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=17, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [12]:
#CELL 18
#Calculate ROC AUC Score 
yvalid_preds= lgb_clf.predict(X_valid)
print ('ROC AUC Score on the test dataset', roc_auc_score (y_valid, yvalid_preds))

ROC AUC Score on the test dataset 0.5160818458998756


In [ ]:
#CELL 19
#TO DO: Determine threshold

#yvalid_preds = lgb_clf.predict_proba(X_valid)[:, 1]
#yvalid_preds_threshold = [1 if pred > 0.10 else 0 for pred in yvalid_preds]
#yvalid_preds= lgb_clf.predict(X_valid)

In [ ]:
#CELL 20
#Visualize the ROC AUC Curve 
#Reference: https://www.kaggle.com/jomaxx/area-under-the-roc-curve-explained 

#calculate false positive rate and true positive rate for all thresholds of the classification
fpr,tpr, thresholds= roc_curve(y_valid, yvalid_preds) 
roc_auc=auc(fpr,tpr)

#Plot ROC 
plt.plot(fpr, tpr, lw=1, label='(AUC = %0.2f)' % (roc_auc))

plt.title('LGB Classifier Receiver Operating Characteristic ')
plt.legend(loc="lower right")
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

In [ ]:
#CELL 21 
#Code to write to a file and save it in the S3: bucket 
#from io import StringIO # python3; python2: BytesIO 
#import boto3

#bucket = "data-science-group12"
#fileName = "test.csv"

#csv_buffer = StringIO()
#test_df.to_csv(csv_buffer)
#s3_resource = boto3.resource('s3')
#s3_resource.Object(bucket, fileName).put(Body=csv_buffer.getvalue())

In [ ]:
#CELL 22
#Previous: ROC AUC Score on the test dataset 0.5000517792764566
#ROC AUC Score on the test dataset 0.5164656366487126
#ROC AUC Score on the test dataset 0.5160818458998756

In [ ]:
#CELL 23
#NOTEBOOK PART 3 : LEARNING GROUP BY ON PREVIOUS APPLICATION

In [6]:
#CELL 24
del train_df 
gc.collect()

csv_df=filteredApps

In [12]:
#CELL 25
groups=csv_df.groupby('SK_ID_CURR')
groups.first()

,SK_ID_PREV,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1038818,Consumer loans,9251.775,179055.000,179055.0,0.000,179055.000,SATURDAY,9,Y,...,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
100003,1810518,Cash loans,98356.995,900000.000,1035882.0,0.000,900000.000,FRIDAY,12,Y,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
100004,1564014,Consumer loans,5357.250,24282.000,20106.0,4860.000,24282.000,FRIDAY,5,Y,...,Connectivity,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0
100006,2827850,Revolving loans,2482.920,0.000,0.0,2693.340,26912.340,THURSDAY,15,Y,...,XNA,12.0,XNA,Card Street,365243.0,-545.0,-215.0,-425.0,-416.0,0.0
100007,2001242,Cash loans,16509.600,180000.000,180000.0,3105.000,180000.000,SUNDAY,14,Y,...,Consumer electronics,18.0,high,Cash Street: high,365243.0,-834.0,-324.0,-354.0,-347.0,0.0
100008,1757431,Cash loans,12144.285,0.000,0.0,12145.500,121455.000,FRIDAY,18,Y,...,XNA,10.0,XNA,Cash,365243.0,-2299.0,-2029.0,-2029.0,-703.0,0.0
100009,2421381,Consumer loans,7435.845,47160.000,39721.5,9432.000,47160.000,SUNDAY,11,Y,...,Consumer electronics,6.0,middle,POS household with interest,365243.0,-795.0,-645.0,-645.0,-636.0,0.0
100010,2349489,Consumer loans,27463.410,247212.000,260811.0,0.000,247212.000,TUESDAY,16,Y,...,Furniture,10.0,low_action,POS industry without interest,365243.0,-1039.0,-769.0,-769.0,-762.0,0.0
100011,1430602,Consumer loans,14614.335,135931.500,134446.5,13594.500,135931.500,MONDAY,17,Y,...,Consumer electronics,12.0,high,POS household with interest,365243.0,-2477.0,-2147.0,-2147.0,-1186.0,1.0


In [18]:
#CELL 26
groups.get_group(456253)['PRODUCT_COMBINATION']

181558     POS mobile with interest
1246881    POS mobile with interest
Name: PRODUCT_COMBINATION, dtype: object

In [20]:
#CELL 27
groups.get_group(456255)

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,CHANNEL_TYPE,CNT_PAYMENT,CODE_REJECT_REASON,DAYS_DECISION,DAYS_FIRST_DRAWING,...,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL,NFLAG_LAST_APPL_IN_DAY,PRODUCT_COMBINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,SK_ID_PREV,WEEKDAY_APPR_PROCESS_START
729432,2250.000,45000.0,45000.0,NaN,45000.0,Country-wide,0.0,HC,-456,NaN,...,XNA,NaN,1,Card Street,NaN,NaN,NaN,20,1708056,SUNDAY
453227,11314.170,82404.0,93298.5,9000.0,82404.0,Country-wide,12.0,XAP,-456,365243.0,...,high,0.0,1,POS mobile with interest,0.095816,NaN,NaN,50,1359084,SUNDAY
608510,16400.610,229500.0,254340.0,NaN,229500.0,Credit and cash offices,24.0,XAP,-648,365243.0,...,middle,1.0,1,Cash X-Sell: middle,NaN,NaN,NaN,-1,2073384,THURSDAY
214743,11090.835,102037.5,112815.0,0.0,102037.5,Regional / Local,12.0,XAP,-991,365243.0,...,middle,0.0,1,POS industry with interest,0.000000,NaN,NaN,500,1743609,THURSDAY
1411592,11514.555,58225.5,58545.0,5823.0,58225.5,Country-wide,6.0,XAP,-500,365243.0,...,high,0.0,1,POS mobile with interest,0.098524,NaN,NaN,20,2729207,FRIDAY
596623,28873.170,450000.0,491580.0,NaN,450000.0,Credit and cash offices,24.0,XAP,-692,365243.0,...,middle,1.0,1,Cash X-Sell: middle,NaN,NaN,NaN,-1,1179690,TUESDAY
814647,30737.655,765000.0,1067940.0,NaN,765000.0,Credit and cash offices,60.0,HC,-171,NaN,...,low_normal,NaN,1,Cash Street: low,NaN,NaN,NaN,-1,1296007,FRIDAY
1383554,54022.140,1170000.0,1271929.5,NaN,1170000.0,AP+ (Cash loan),36.0,XAP,-787,365243.0,...,low_normal,0.0,1,Cash X-Sell: low,NaN,NaN,NaN,6,2631384,FRIDAY


In [ ]:
#CELL 28